In [1]:
from tinybig.config import rpn_config
from tinybig.util import set_random_seed

print('loading configs...')
config_file_name = 'feynman_configs'
config_obj = rpn_config()
config = config_obj.load_yaml(cache_dir='./configs', config_file=config_file_name + '.yaml')
print(config)

loading configs...
{'configurations': {'device': 'cpu', 'random_seed': 4567, 'data_configs': {'data_class': 'tinybig.data.feynman_function', 'data_parameters': {'name': 'elementary_function', 'train_batch_size': 100, 'test_batch_size': 100, 'equation_index': 4}}, 'learner_configs': {'learner_class': 'tinybig.learner.backward_learner', 'learner_parameters': {'name': 'error_backward_propagation', 'n_epochs': 2000, 'optimizer_configs': {'optimizer_class': 'torch.optim.AdamW', 'optimizer_parameters': {'lr': 0.003, 'weight_decay': 0.0001}}, 'lr_scheduler_configs': {'lr_scheduler_class': 'torch.optim.lr_scheduler.ExponentialLR', 'lr_scheduler_parameters': {'gamma': 0.999}}, 'loss_configs': {'loss_class': 'torch.nn.MSELoss', 'loss_parameters': {'reduction': 'mean'}}}}, 'model_configs': {'model_class': 'tinybig.model.rpn', 'model_parameters': {'name': 'reconciled_polynomial_network', 'depth': 3, 'depth_alloc': [1, 1, 1], 'layer_configs': [{'layer_class': 'tinybig.module.rpn_layer', 'layer_para

In [2]:
print('setting up environments...')
device = config['configurations'].pop('device')
random_seed = config['configurations'].pop('random_seed')
set_random_seed(random_seed)
print('device: ', device, '; random_seed: ', random_seed)

setting up environments...
device:  cpu ; random_seed:  4567


In [3]:
print('instantiating objects from config...')
object_dict = config_obj.instantiate_object_from_config(config['configurations'])
data_obj, model_obj, learner_obj, metric_obj, result_obj = [object_dict[name] for name in ['data', 'model', 'learner', 'metric', 'result']]
print('parameter num: ', sum([parameter.numel() for parameter in model_obj.parameters()]))

instantiating objects from config...
parameter num:  313


In [4]:
print('loading dataset...')
data_loader = data_obj.load()
print(data_loader['str_equation'])

loading dataset...
I.9.18,5,F,G*m1*m2/((x2-x1)**2+(y2-y1)**2+(z2-z1)**2),9,m1,1,2,m2,1,2,G,1,2,x1,3,4,x2,1,2,y1,3,4,y2,1,2,z1,3,4,z2,1,2,,,


In [5]:
print('training model...')
training_records = learner_obj.train(model=model_obj, data_loader=data_loader, device=device,
                                     metric=metric_obj, disable_tqdm=True, display_step=100)
model_obj.save_ckpt(cache_dir='./ckpt', checkpoint_file=f'{config_file_name}_checkpoint')

training model...
Epoch: 0, Test Loss: 0.11375603005290032, Test Score: 0.11375603045499631, Time Cost: 0.012421131134033203
Epoch: 100, Test Loss: 0.00030789836309850214, Test Score: 0.000307898367222146, Time Cost: 0.010743141174316406
Epoch: 200, Test Loss: 0.00023267639917321504, Test Score: 0.00023267639499684217, Time Cost: 0.011265993118286133
Epoch: 300, Test Loss: 0.00026345557707827536, Test Score: 0.00026345557328203247, Time Cost: 0.012063026428222656
Epoch: 400, Test Loss: 0.00013252994176582433, Test Score: 0.00013252994637558585, Time Cost: 0.011242151260375977
Epoch: 500, Test Loss: 0.0001221301812620368, Test Score: 0.00012213017811540662, Time Cost: 0.012245893478393555
Epoch: 600, Test Loss: 0.0001262327248696238, Test Score: 0.00012623272414215736, Time Cost: 0.010609149932861328
Epoch: 700, Test Loss: 0.0001131150889705168, Test Score: 0.00011311509136536863, Time Cost: 0.010414838790893555
Epoch: 800, Test Loss: 9.948268307198304e-05, Test Score: 9.948268467315169

In [6]:
print('testing model...')
test_result = learner_obj.test(model=model_obj, test_loader=data_loader['test_loader'], device=device,
                               metric=metric_obj)

testing model...


In [7]:
print('evaluating result...')
print(metric_obj.__class__.__name__, metric_obj.evaluate(y_true=test_result['y_true'], y_pred=test_result['y_pred'], y_score=test_result['y_score'], ))

evaluating result...
mse 5.7441863398028705e-05


In [8]:
print('saving result...')
result_obj.save(test_result, cache_dir='./result', output_file='{}_result'.format(config_file_name))
result_obj.save(training_records, cache_dir='./result', output_file='{}_record'.format(config_file_name))

saving result...
